In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# 结巴分词
import jieba
# 数据总数
from sklearn.feature_extraction.text import CountVectorizer
# 朴素贝叶斯
from sklearn.naive_bayes import MultinomialNB

# 获取数据

In [4]:
data = pd.read_csv("../data/书籍评价.csv", encoding="gbk")
data.head()

,Unnamed: 0,内容,评价
0,0,从编程小白的角度看，入门极佳。,好评
1,1,很好的入门书，简洁全面，适合小白。,好评
2,2,讲解全面，许多小细节都有顾及，三个小项目受益匪浅。,好评
3,3,前半部分讲概念深入浅出，要言不烦，很赞,好评
4,4,看了一遍还是不会写，有个概念而已,差评


# 数据基本处理
## 取出内容列，对数据进行分析

In [5]:
content = data['内容']
content.head()

0              从编程小白的角度看，入门极佳。
1            很好的入门书，简洁全面，适合小白。
2    讲解全面，许多小细节都有顾及，三个小项目受益匪浅。
3          前半部分讲概念深入浅出，要言不烦，很赞
4             看了一遍还是不会写，有个概念而已
Name: 内容, dtype: object

## 判定评判标准 -- 1好评;0差评

In [6]:
data.loc[:, '评价'] == '好评'

0      True
1      True
2      True
3      True
4     False
5     False
6     False
7     False
8     False
9      True
10    False
11    False
12    False
Name: 评价, dtype: bool

In [7]:
# 增加一列表示好评和差评,下面没有使用,直接使用的好评和差评
data.loc[data.loc[:, '评价'] == '好评', '评论称号'] = 1
data.loc[data.loc[:, '评价'] == '差评', '评论称号'] = 0
data.head()

,Unnamed: 0,内容,评价,评论称号
0,0,从编程小白的角度看，入门极佳。,好评,1.0
1,1,很好的入门书，简洁全面，适合小白。,好评,1.0
2,2,讲解全面，许多小细节都有顾及，三个小项目受益匪浅。,好评,1.0
3,3,前半部分讲概念深入浅出，要言不烦，很赞,好评,1.0
4,4,看了一遍还是不会写，有个概念而已,差评,0.0


## 加载停用词

In [19]:
stopwords = []
with open('../data/stopwords.txt', 'r', encoding='utf-8') as f:
    # 取出所有行
    lines = f.readlines()
    #print(lines)
    for temp in lines:
        # 取出换行
        line = temp.strip()
        #print(line)
        stopwords.append(line)
# set去重
stopwords = list(set(stopwords))
stopwords

['',
 '积极',
 '挨门挨户',
 '呵',
 '［②',
 '届时',
 '老大',
 '除却',
 '还要',
 '需要',
 '岂但',
 '任凭',
 '不可开交',
 '使',
 '［③ｇ］',
 '九',
 '并无',
 '前面',
 '加上',
 '啊',
 '接著',
 '隔夜',
 '彼',
 '不管怎样',
 '＜Δ',
 '以前',
 '结果',
 '概',
 '可好',
 '［①Ｅ］',
 '进步',
 '>>',
 '粗',
 '在下',
 '饱',
 '最',
 '不了',
 '往',
 '一来',
 '良好',
 '类如',
 '一方面',
 '几度',
 '将',
 '＆',
 '设若',
 '不胜',
 '颇',
 '①',
 '第',
 '凑巧',
 '连',
 '保持',
 '遵照',
 '您',
 '按照',
 '打',
 '那些',
 '就此',
 '引起',
 '＠',
 '傥然',
 '三番五次',
 '诚如',
 '何乐而不为',
 '现代',
 '以至于',
 '其它',
 '〉',
 '什么',
 '即是说',
 '怎麽',
 '现在',
 '［②⑩］',
 '严格',
 '你',
 '活',
 '怪不得',
 '吧哒',
 '也好',
 '普遍',
 '归根到底',
 '逢',
 '［⑤］',
 '喽',
 '原来',
 '否则',
 '他人',
 '［②ｊ］',
 '接着',
 '～',
 '［①ｉ］',
 '不可抗拒',
 '自己',
 '后来',
 '不敢',
 '窃',
 '或',
 '另行',
 '`',
 '说说',
 '几经',
 '半',
 '假如',
 '如此',
 '自从',
 '沿着',
 '起来',
 '这儿',
 '遵循',
 '充其量',
 '带',
 '［①⑤］',
 '三番两次',
 '－［＊］－',
 '刚好',
 '共总',
 '０：２',
 '以下',
 '［①⑨］',
 '看到',
 '［①ｄ］',
 '吧',
 '或多或少',
 '接连不断',
 '一片',
 '匆匆',
 '如何',
 '恰恰相反',
 '有的是',
 '莫不',
 '何苦',
 '老是',
 '嘎',
 '７',
 '亲手',
 '⑨',
 '进来',
 '屡屡',
 '她的',
 '归',


## 把“内容”处理，转化成标准格式

In [24]:
comment_list = []
for temp in content:
    # 对文本数据进行切割
    # cut_all 参数默认为 False, 不一次性切割所有, 所有使用 cut 方法时默认为精确模式 
    seg_list = jieba.cut(temp, cut_all=False)
    #print(seg_list)    # 返回对象 <generator object Tokenizer.cut at 0x0000000007CF7DB0>
    # 使用逗号分隔成字符串
    seg_str = ','.join(seg_list)
    #print(seg_str)
    # 转化成列表形式
    comment_list.append(seg_str)
comment_list 

[' ,从,编程,小白,的,角度看,，,入门,极佳,。',
 '很,好,的,入门,书,，,简洁,全面,，,适合,小白,。',
 '讲解,全面,，,许多,小,细节,都,有,顾及,，,三个,小,项目,受益匪浅,。',
 '前半部,分讲,概念,深入浅出,，,要言不烦,，,很赞',
 '看,了,一遍,还是,不会,写,，,有个,概念,而已',
 '中规中矩,的,教科书,，,零,基础,的,看,了,依旧,看不懂',
 '内容,太,浅显,，,个人,认为,不,适合,有,其它,语言,编程,基础,的,人',
 '破书,一本',
 '适合,完完全全,的,小白读,，,有,其他,语言,经验,的,可以,去,看,别的,书',
 '基础知识,写,的,挺,好,的,！',
 '太,基础',
 '略,_,嗦,。,。,适合,完全,没有,编程,经验,的,小白',
 '真的,真的,不,建议,买']

## 统计词的个数

In [26]:
# 进行统计词个数
# 实例化对象
# CountVectorizer 类会将文本中的词语转换为词频矩阵
con = CountVectorizer(stop_words=stopwords)
# 进行词数统计
# 它通过 fit_transform 函数计算各个词语出现的次数
x = con.fit_transform(comment_list)
# 0 1 是一句话包含哪一些词
x.toarray()

d:\anaconda3\envs\ai\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0

In [27]:
# 通过 get_feature_names()可获取词袋中所有文本的关键字
name = con.get_feature_names()  
name

['一本',
 '一遍',
 '三个',
 '中规中矩',
 '依旧',
 '入门',
 '内容',
 '分讲',
 '前半部',
 '受益匪浅',
 '基础',
 '基础知识',
 '完完全全',
 '小白',
 '小白读',
 '建议',
 '很赞',
 '教科书',
 '有个',
 '极佳',
 '概念',
 '浅显',
 '深入浅出',
 '看不懂',
 '真的',
 '破书',
 '简洁',
 '细节',
 '经验',
 '编程',
 '要言不烦',
 '角度看',
 '讲解',
 '语言',
 '适合',
 '项目',
 '顾及']

## 准备训练集和测试集

In [36]:
# 准备训练集   这里将文本前10行当做训练集  后3行当做测试集
x_train = x.toarray()[:10, :]
y_train = data['评价'][:10]

# 准备测试集
x_test = x.toarray()[10:, :]
y_test = data['评价'][10:]
y_train, y_test

(0    好评
 1    好评
 2    好评
 3    好评
 4    差评
 5    差评
 6    差评
 7    差评
 8    差评
 9    好评
 Name: 评价, dtype: object,
 10    差评
 11    差评
 12    差评
 Name: 评价, dtype: object)

# 模型训练

In [38]:
# 构建⻉叶斯算法分类器
mb = MultinomialNB(alpha=1)  # alpha 为可选项，默认 1.0，添加拉普拉修/Lidstone 平滑参数
# 训练数据
mb.fit(x_train, y_train)
# 预测数据
y_predict = mb.predict(x_test)
# 预测值与真实值展示
print('预测值：', y_predict)
print('*' * 50)
print('真实值：', y_test)

预测值： ['差评' '差评' '差评']
**************************************************
真实值： 10    差评
11    差评
12    差评
Name: 评价, dtype: object


# 模型评估

In [34]:
mb.score(x_test, y_test)

1.0